
# Make sure to edit the first cell with the appropriate path

In [ ]:
import sys, os

#change the path to the directory that ctmc.py is located in on your system
sys.path.append(os.path.expanduser('~/source/discrete_states/'))

In [ ]:
from ctmc import ContinuousTimeMarkovChain as MC
from ctmc import normal_generator, gamma_generator, uniform_generator, cyclic_generator, detailed_balance_generator, arrhenius_pump_generator

import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
#make 1 machines with 5 states, generated using the "cyclic" generator fo transition rates
#machine = MC(generator=cyclic_generator, S=5, N=1, max_jump=2)

#make 1 machines with 5 states, generated using the arrhenious pump generator. Here, we select 3 transitions to be artifically pumped 
# beyond the normal DB transitions the arrheniosu equation gives. The pump strength is negative to be in the regime where NESS and MEPS can be different 
machine = MC(generator=arrhenius_pump_generator, S=5, N=1, n_pumps=3, pump_strength=-10)


In [ ]:
# the cornerstone is the rate matrix, note it is scaled so the fastest rate value will be -1 always, this scaling can be set with machine.timescale 
# this basically sets a uniform timescale for different models based on the state with the most outgoing activity
print(machine.R)
#print(machine.rev_R)
# this scaling is kept track of with the machine.scale attribute
machine.scale

In [ ]:
# find and the NESS state
ness = machine.get_ness()
# finds and the minimum entropy producing state
meps = machine.get_meps()
# returns a uniform state
unif = machine.get_uniform()


In [ ]:
# for states, we can grab a variety of values, like the entropy produciton rate
print('epr:')
print(machine.get_epr(ness), machine.get_epr(meps), machine.get_epr(unif))
#activity (which is the sum of all (positive) transition rates)
print('activity:')
print(machine.get_activity(ness), machine.get_activity(meps), machine.get_activity(unif))
# or probability current (which is the total amount of probability flowing between states, note it is zero for the NESS state)
print('current:')
print(machine.get_prob_current(ness), machine.get_prob_current(meps), machine.get_prob_current(unif))

In [ ]:
print(machine.get_prob_current(ness), machine.get_prob_current(meps), machine.get_prob_current(unif))

In [ ]:
# returns a random state drawn from a uniform distrbution over all states
random = machine.get_random_state()

# returns a "localized" state which is basically a guassian centered on a particular state, putting the states on a ring, default is random peak and variance
random_local = machine.get_local_state()
# you can put in manual arguments too, though theres an annoying input issue with variances for now, requiring the array of a list input
hardcoded_local_1 = machine.get_local_state(mu=2.6, sigma= np.array([.75]))
hardcoded_local_2 = machine.get_local_state(mu=2.6, sigma= np.array([1.5]))


fig, ax = plt.subplots(figsize=(10,6))
states = [random, random_local, hardcoded_local_1, hardcoded_local_2]
labels = ['rand','rand_loc', 'local_1', 'local_2']
for s,l in zip(states, labels):
    ax.plot(range(machine.S), s.T, label=l, marker='o')
ax.set_xlabel('state #');
ax.set_ylabel('p(state #)');
fig.legend()


In [ ]:
# and, we can evolve these states according to the rate matrix:
test_state = machine.get_uniform()
meps_state = meps

T = 25
dt = .2

states = [test_state]
meps_states = [meps_state]

for i in range(T):
    test_state = machine.evolve_state(test_state, dt=dt)
    meps_state = machine.evolve_state(meps_state, dt=dt)
    states.append(test_state)
    meps_states.append(meps_state)
    

fig, ax = plt.subplots(1,2, figsize=(10,5))

ax[0].hlines(ness, xmin=0, xmax=T, colors='k', linestyle='--', label='ness')
ax[1].hlines(ness, xmin=0, xmax=T, colors='k', linestyle='--')

ax[0].plot(np.squeeze(states))
ax[1].plot(np.squeeze(meps_states),label=[f'$s_{i}$' for i in range(test_state.size)])
ax[0].set_xlabel('$t$')
ax[0].set_ylabel('$p_(s_i(t))$')

ax[0].set_title('test_state_dynamics');
ax[1].set_title('meps_state_dynamics');
fig.legend();

In [ ]:
# all of these things also work with ensembles of machines, for example...

# make an ensemble of 100 machines with 50 states each, generated using the default uniform generator for transition rates
#machines = MC(S=50, N=100)

# or whatever generator we want, here is an example of the arrhenious pump
machines = MC(generator=arrhenius_pump_generator, S=20, N=100, n_pumps=50, pump_strength=-12)



In [ ]:
%%time

ness_100 = machines.get_ness()
meps_100 = machines.get_meps()
unif_100 = machines.get_uniform()

ness_epr = machines.get_epr(ness_100)
meps_epr = machines.get_epr(meps_100)
unif_epr = machines.get_epr(unif_100)


ness_activity = machines.get_activity(ness_100)
meps_activity = machines.get_activity(meps_100)
unif_activity = machines.get_activity(unif_100)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,7))
ax[0].scatter(range(100), unif_epr-meps_epr, label='unif', alpha=.8)
ax[0].scatter(range(100), ness_epr-meps_epr, label='ness', alpha=.8)

ax[1].scatter(range(100), unif_activity)
ax[1].scatter(range(100), ness_activity)
ax[1].scatter(range(100), meps_activity, label='meps')

ax[0].set_title('EPR- MEPR')
ax[1].set_title('activity')

fig.legend()

In [ ]:
%%time
# example of doing a sweep over many different kind of generators with different S and N, and saving some of the data. The NESS calculating will become numeric if the matrices get
# too large to invert exactly
SS = [3, 6, 20, 100, 250]
trials = [300, 300, 300, 300, 300]

normal_output, uniform_output, gamma_output, cyclic_output, balanced_output = {}, {}, {}, {}, {}

for a in range(5):
    print(a)
    output = [normal_output, uniform_output, gamma_output, cyclic_output, balanced_output ][a]
    gen = [normal_generator, uniform_generator, gamma_generator, cyclic_generator, detailed_balance_generator][a]
    
    
    for s,trial in zip(SS,trials):
        p = MC(S=s, N=trial, generator=gen)
        #R = [ np.random.uniform(0,1,(S,S)) for i in range(trials)]
        #for p,r in zip(procs,R):
        #    p.set_rate_matrix(r, max_rate=1)   
        dct = {'ness':p.get_epr(p.get_ness()), 
               'meps':p.get_epr(p.get_meps()), 
               'unif':p.get_epr(p.get_uniform()),
               'rand':p.get_epr(p.get_random_state()),
               'local':p.get_epr(p.get_local_state()),
               'N':trial, 'scale':p.scale, 
               'nm_dkl':p.dkl(p.ness,p.meps),'mn_dkl':p.dkl(p.meps,p.ness)}
        
        output[f'{p.S}'] = dct


defaulting to numeric solution
2


In [ ]:
# some example data calls


nstates=['3','6', '100']

plot_generator = normal_output

fig,axs = plt.subplots(1,len(nstates), sharey=True, sharex=True)


state_types = ['local','ness','rand']

for ax, n in zip(axs, nstates):

    for state_type in state_types:
        meps_epr = plot_generator[n]['meps']
        ax.scatter(meps_epr, plot_generator[n][state_type]-meps_epr, label=state_type)

    ax.set_title(f'N={n}')
    ax.legend()
    ax.set_xlabel('MEPS EPR')
    ax.set_ylabel('EPR-MEPS_EPR')

    ax.set_yscale('log')


